<a href="https://colab.research.google.com/github/rrwiren/ilmanlaatu-ennuste-helsinki/blob/main/PIPELINE_v0.7_FB-PROPHET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
==========================================================================================
 PIPELINE v0.7 - PROPHET-MALLIN KOULUTUS JA EVALUOINTI (Korjattu versionhaku)
==========================================================================================

 Versio: 0.7 (Korjattu versionhaku)
 Päivitetty: 2025-04-15 15:02 (EEST)
 Edellinen versio: 0.6 (XGBoost Pipeline)

 Skriptin tarkoitus: <Sama kuin aiemmin>
 Vaaditut kirjastot: <Samat kuin ennen>
"""

# === 1. Alustus ja Kirjastojen Tuonti ===
print("="*60); print(" Vaihe 1: Alustus ja Kirjastojen Tuonti"); print("="*60)
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
import joblib # Pidetään mukana, vaikkei tässä käytetä

# Tarkistetaan ja tuodaan Prophet ja sen versio (KORJATTU)
try:
    # Tuodaan Prophet-luokka suoraan käyttöön
    from prophet import Prophet
    # Tuodaan myös itse kirjasto version tarkistusta varten
    import prophet as prophet_module
    # Haetaan versio moduulista, ei luokasta
    print(f"--- Käytössä oleva Prophet-versio: {prophet_module.__version__} ---")
except ImportError:
    print("VAROITUS: 'prophet'-kirjastoa ei löydy.")
    print("Asenna se komennolla: pip install prophet")
    Prophet = None # Estetään jatko, jos import epäonnistuu

# Tarkistetaan ja tuodaan PyArrow
try: import pyarrow; print("'pyarrow' löytyy.")
except ImportError: print("VAROITUS: 'pyarrow' puuttuu."); pyarrow = None

# Tuodaan muut
import requests; import io; import seaborn as sns;
plt.style.use('ggplot')

# Tulostetaan myös XGBoost-versio, jos se on asennettu
try: import xgboost as xgb; print(f"--- Käytössä oleva XGBoost-versio: {xgb.__version__} ---")
except ImportError: pass # Ohitetaan hiljaa, jos ei löydy


# --- Konfiguraatio ---
INPUT_DATA_PATH = "https://raw.githubusercontent.com/rrwiren/ilmanlaatu-ennuste-helsinki/main/data/processed/Helsinki_Data_With_Features_Pipeline_v0.5.parquet"
print(f"Käytetään dataa URL-osoitteesta: {INPUT_DATA_PATH}")
TARGET_VARIABLE = 'Ozone'; TEST_DATA_RATIO = 0.2

# === 2. Datan Lataus ===
def lataa_esikasitelty_data(url):
    print("\n" + "="*60); print(" Vaihe 2: Datan Lataus"); print("="*60)
    if not pyarrow: print("VIRHE: pyarrow puuttuu."); return None
    print(f"Yritetään ladata dataa osoitteesta: {url}")
    df = None
    try:
        df = pd.read_parquet(url)
        print(f"Data ladattu."); print(f"Muoto: {df.shape}")
        if not isinstance(df.index, pd.DatetimeIndex):
            print("VAROITUS: Indeksi ei ole DatetimeIndex. Muunnetaan...");
            df.index = pd.to_datetime(df.index)
        return df
    except Exception as e: print(f"VIRHE datan latauksessa: {e}"); return None

# === 3. Datan Valmistelu Prophetille ===
# KORJATTU: Tunnistaa ja nimeää 'ds' ja 'y' sarakkeet oikein
def valmistele_data_prophetille(df, target_sarake):
    """Muuntaa DataFramen Prophetin vaatimaan muotoon (ds, y)."""
    print("\n" + "="*60); print(" Vaihe 3: Datan Valmistelu Prophetille"); print("="*60)
    if df is None or df.empty: print("VIRHE: Ei dataa valmisteltavaksi."); return None
    if target_sarake not in df.columns: print(f"VIRHE: Kohde '{target_sarake}' puuttuu."); return None
    if not isinstance(df.index, pd.DatetimeIndex): print("VIRHE: Indeksi ei DatetimeIndex."); return None
    print(f"Valmistellaan data target-sarakkeelle: '{target_sarake}'")
    df_prophet = df[[target_sarake]].copy().reset_index()
    print(f"  -> Sarakkeet reset_indexin jälkeen: {df_prophet.columns.tolist()}")
    col_names = df_prophet.columns; original_index_col_name = None
    if 'Timestamp' in col_names: original_index_col_name = 'Timestamp'
    elif 'index' in col_names: original_index_col_name = 'index'
    else:
        potential_ds_cols = col_names.difference([target_sarake])
        if len(potential_ds_cols) == 1: original_index_col_name = potential_ds_cols[0]; print(f"  -> Oletetaan aikaleimasarakkeen olevan: {original_index_col_name}")
        else: print(f"VIRHE: Ei voitu tunnistaa aikaleimasaraketta sarakkeista {col_names.tolist()}"); return None
    try:
        df_prophet = df_prophet.rename(columns={original_index_col_name: 'ds', target_sarake: 'y'})
        if not all(col in df_prophet.columns for col in ['ds', 'y']): raise ValueError("Uudelleennimeäminen epäonnistui")
        print("Data valmisteltu Prophetille (sarakkeet 'ds' ja 'y')."); print(df_prophet.head())
        return df_prophet[['ds', 'y']]
    except Exception as rename_e: print(f"VIRHE sarakkeiden uudelleennimeämisessä: {rename_e}"); return None

# === 4. Datan Jako (Prophet) ===
def jaa_data_prophet(df_prophet, test_osuus):
    """Jakaa Prophet-muotoisen datan kronologisesti."""
    print("\n" + "="*60); print(" Vaihe 4: Datan Jako (Prophet)"); print("="*60)
    if df_prophet is None or df_prophet.empty: print("VIRHE: Ei dataa."); return None, None
    n=len(df_prophet); split_index=int(n*(1-test_osuus));
    if split_index<=0 or split_index>=n: print(f"VIRHE: Testikoko {test_osuus} virheellinen."); return None,None
    df_train = df_prophet.iloc[:split_index]; df_test = df_prophet.iloc[split_index:]
    print(f"Data jaettu: Opetus {len(df_train)} riviä ({df_train['ds'].min()} - {df_train['ds'].max()})")
    print(f"            Testi  {len(df_test)} riviä ({df_test['ds'].min()} - {df_test['ds'].max()})")
    return df_train, df_test

# === 5. Prophet-mallin Koulutus ===
def kouluta_prophet_malli(df_train):
    """Alustaa ja kouluttaa Prophet-mallin."""
    print("\n" + "="*60); print(" Vaihe 5: Prophet-mallin Koulutus"); print("="*60)
    if Prophet is None: print("VIRHE: Prophet-kirjastoa ei ole asennettu."); return None
    if df_train is None or df_train.empty: print("VIRHE: Opetusdata puuttuu."); return None
    print("Alustetaan Prophet-malli oletusparametreilla...")
    model = Prophet()
    try: print("Aloitetaan mallin sovittaminen..."); model.fit(df_train); print("Prophet-malli koulutettu."); return model
    except Exception as e: print(f"VIRHE Prophet-mallin koulutuksessa: {e}"); return None

# === 6. Ennustaminen ja Evaluointi ===
def ennusta_ja_evaluoi_prophet(model, df_test):
    """Tekee ennusteen testijaksolle ja evaluoi mallin."""
    print("\n" + "="*60); print(" Vaihe 6: Ennustaminen ja Evaluointi"); print("="*60)
    if model is None or df_test is None or df_test.empty: print("VIRHE: Malli tai testidata puuttuu."); return None
    future = df_test[['ds']].copy(); print("Tehdään ennuste testijaksolle...");
    try: forecast = model.predict(future); print("Ennuste laskettu."); print("Ennusteen loppuosa:"); print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())
    except Exception as e: print(f"VIRHE ennusteen laskemisessa: {e}"); return None
    eval_df = pd.merge(df_test[['ds', 'y']], forecast[['ds', 'yhat']], on='ds')
    y_true = eval_df['y']; y_pred = eval_df['yhat']
    mae = mean_absolute_error(y_true, y_pred); rmse = np.sqrt(mean_squared_error(y_true, y_pred)); r2 = r2_score(y_true, y_pred)
    print(f"\nSuorituskykymetriikat (Testisetti):"); print(f"  MAE:  {mae:.3f}"); print(f"  RMSE: {rmse:.3f}"); print(f"  R²:   {r2:.3f}")
    print("\nVisualisoidaan ennuste...");
    try:
        fig1 = model.plot(forecast); plt.plot(eval_df['ds'], eval_df['y'], '.r', alpha=0.5, label='Todelliset (Testi)')
        plt.title('Prophet Ennuste vs. Todelliset'); plt.xlabel('Aika'); plt.ylabel(TARGET_VARIABLE)
        plt.legend(loc='upper left'); plt.show()
        print("\nVisualisoidaan mallin komponentit...");
        fig2 = model.plot_components(forecast); plt.show()
    except Exception as plot_e: print(f"VIRHE visualisoinnissa: {plot_e}")
    return {'MAE': mae, 'RMSE': rmse, 'R2': r2}

# === Pääsuorituslohko ===
if __name__ == "__main__":
    print("\n" + "="*80); print(" ALOITETAAN PROPHET PIPELINE v0.7 (Korjattu versionhaku)"); print("="*80)
    df_original, df_prophet_format = None, None; df_train, df_test = None, None
    prophet_model, prophet_results = None, None

    if Prophet is None or not pyarrow:
         print("\nPipeline keskeytetään. Vaadittuja kirjastoja (Prophet tai PyArrow) puuttuu.")
    else:
        try:
            df_original = lataa_esikasitelty_data(INPUT_DATA_PATH)
            if df_original is None: raise ValueError("Datan lataus epäonnistui.")
            df_prophet_format = valmistele_data_prophetille(df_original, TARGET_VARIABLE)
            if df_prophet_format is None: raise ValueError("Datan valmistelu Prophetille epäonnistui.")
            df_train, df_test = jaa_data_prophet(df_prophet_format, TEST_DATA_RATIO)
            if df_train is None: raise ValueError("Datan jako epäonnistui.")
            prophet_model = kouluta_prophet_malli(df_train)
            if prophet_model is None: raise ValueError("Prophet-mallin koulutus epäonnistui.")
            prophet_results = ennusta_ja_evaluoi_prophet(prophet_model, df_test)
            if prophet_results is None: raise ValueError("Prophet-mallin evaluointi epäonnistui.")

            # Vertailu aiempiin malleihin (Lisää v0.5 ja v0.6 tulokset)
            print("\nHaetaan aiempien mallien tulokset vertailua varten...")
            baseline_mae = 12.804; baseline_rmse = 16.872; baseline_r2 = 0.331   # KORVAA OIKEILLA!
            xgboost_mae = 8.807; xgboost_rmse = 10.764; xgboost_r2 = 0.565     # KORVAA OIKEILLA! (v0.6 no_es -ajosta)
            if baseline_mae is not None and xgboost_mae is not None and prophet_results is not None:
                print("\n" + "-"*60); print(" Vertailu aiempiin malleihin"); print("-"*60)
                print(f"Metriikka        | Baseline (LR) | XGBoost (v0.6) | Prophet (v0.7)")
                print(f"-----------------|---------------|----------------|----------------")
                print(f"MAE              | {baseline_mae:>13.3f} | {xgboost_mae:>14.3f} | {prophet_results['MAE']:>14.3f}")
                print(f"RMSE             | {baseline_rmse:>13.3f} | {xgboost_rmse:>14.3f} | {prophet_results['RMSE']:>14.3f}")
                print(f"R²               | {baseline_r2:>13.3f} | {xgboost_r2:>14.3f} | {prophet_results['R2']:>14.3f}")
                print("-"*60)
            else: print("\nAiempien mallien tai Prophetin tuloksia puuttuu, vertailua ei tehdä täysin.")

        except Exception as e:
            print("\n" + "!"*80); print(f" KRIITTINEN VIRHE PIPELINEN SUORITUKSESSA: {e}");
            import traceback; traceback.print_exc(); print("!"*80)
        finally:
            print("\n" + "="*80); print(" PROPHET PIPELINE SUORITETTU LOPPUUN"); print("="*80)